# Modeling - Given these features, what model works best?

## Here you can actually test assumptions
1. Linearity by residuals vs predicted plot.
2. Homoscedasticity with spread of residuals vs predicted.
3. Normality of errors with Q-Q plot of residuals.
4. Independence with residuals vs time/order.
5. Multicollinearity with VIF, condition number.
6. Influential poitns with Cook's distance.

## They require predictions, residuals, fitted model parameters so they belong to modeling.ipynb.

In [ ]:
# 3rd party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# linear_regression_001
from linear_regression_001.data.loader import load_processed_data
from linear_regression_001.features.build_features import split_features_target, build_features

# pipeline
from sklearn.model_selection import train_test_split

In [ ]:
df = load_processed_data("processed_insurance.csv")